In [1]:
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool



In [43]:
def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""

    connection = sqlite3.connect('../players.db', check_same_thread=False)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )

In [45]:
engine = get_engine_for_chinook_db()
db = SQLDatabase(engine)
sqlite_uri = 'sqlite:///./players.db' 
db = SQLDatabase.from_uri(sqlite_uri)


In [46]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [47]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [48]:
toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x13244ff70>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x13244ff70>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x13244ff70>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [49]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)

In [50]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

/Users/manojkr/Development/miniforge3/envs/valorant/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


['dialect', 'top_k']


In [51]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)
system_message

'System: You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\nTo start you should ALWAYS look at the tables in the datab

In [52]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

In [53]:
example_query = "who are the top 10 igls (role=igl) according to clutch success and kill_assist_trade_survive percentage?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

who are the top 10 igls (role=igl) according to clutch success and kill_assist_trade_survive percentage?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_TF6hpgnHv23gx18BKcrsAhA6)
 Call ID: call_TF6hpgnHv23gx18BKcrsAhA6
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Agents, Players
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_gtLxxoveAJC5ULEbKReijDeZ)
 Call ID: call_gtLxxoveAJC5ULEbKReijDeZ
  Args:
    table_names: Players
  sql_db_schema (call_3Vecr5WxcBpAfB0JZtTY1QsB)
 Call ID: call_3Vecr5WxcBpAfB0JZtTY1QsB
  Args:
    table_names: Agents
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Agents" (
	agent_id INTEGER, 
	player_id INTE

In [57]:
example_query = "create a 5 player team with one igl,  4 normal players, where each player plays one of the following (cypher, raze, omen, sova, jett). make sure the agent was every players’ most played agent. Agents in the team should be unique.  No 2 players can be the same agent.  I need the name of the players, agent and how many times have they played that agent."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

create a 5 player team with one igl,  4 normal players, where each player plays one of the following (cypher, raze, omen, sova, jett). make sure the agent was every players’ most played agent. Agents in the team should be unique.  No 2 players can be the same agent.  I need the name of the players, agent and how many times have they played that agent.
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_imq2ZrAtU35EWdowbEBoELCc)
 Call ID: call_imq2ZrAtU35EWdowbEBoELCc
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Agents, Players
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_onLISZVESXuvUTJt0taYYidM)
 Call ID: call_onLISZVESXuvUTJt0taYYidM
  Args:
    table_names: Agents
  sql_db_schema (call_PNi2gpx5TPsX